# Model inspection <a class="tocSkip">
> After generating a model we inspect it here to find its flaws.

In [ ]:
#default_exp inspection

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
#export
import pandas as pd
from fastai2.tabular.all import *
from fastai2 import metrics
from sklearn import ensemble
import pickle
from src.utils import *
from src.preprocessing import *
from src.modelling import *
from typing import List
import matplotlib.pyplot as plt

In [ ]:
with open(model_data_path, "rb") as f:
    to = pickle.load(f)

In [ ]:
dls = to.dataloaders()

In [ ]:
#hide
dls.show_batch()

,Id,MSSubClass,MSZoning,LotFrontage,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,Heating,HeatingQC,CentralAir,Electrical,LowQualFinSF,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageFinish,GarageCars,GarageQual,GarageCond,PavedDrive,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,SaleType,SaleCondition,LotArea_na,YearBuilt_na,YearRemodAdd_na,MasVnrArea_na,BsmtFinSF2_na,BsmtUnfSF_na,TotalBsmtSF_na,1stFlrSF_na,2ndFlrSF_na,GrLivArea_na,GarageYrBlt_na,GarageArea_na,WoodDeckSF_na,OpenPorchSF_na,LotArea,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,GrLivArea,GarageYrBlt,GarageArea,WoodDeckSF,OpenPorchSF,SalePrice
0,1234,20,RL,#na#,Pave,#na#,IR1,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,1Story,5,5,Hip,CompShg,Plywood,Plywood,BrkFace,TA,TA,CBlock,TA,TA,No,Rec,1000,Unf,GasA,Fa,Y,SBrkr,0,1,0,1,0,3,1,TA,6,Typ,0,#na#,Attchd,RFn,2,TA,TA,Y,0,0,0,0,#na#,MnPrv,#na#,0,5,COD,Abnorml,False,False,False,False,False,False,False,False,False,False,False,False,False,False,12159.999991,1959.000000,1959.000001,179.999998,-2.059479e-08,188.000011,1188.000004,1188.000000,0.000004,1188.000000,1959.000000,531.000000,-0.000003,-9.072535e-07,11.863583
1,142,20,RL,78,Pave,#na#,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,1Story,7,5,Gable,CompShg,VinylSd,VinylSd,None,Gd,TA,PConc,Gd,TA,Av,GLQ,1300,Unf,GasA,Ex,Y,SBrkr,0,1,0,2,0,3,1,Gd,7,Typ,0,#na#,Attchd,Fin,2,TA,TA,Y,0,0,0,0,#na#,#na#,#na#,0,1,WD,Normal,False,False,False,False,False,False,False,False,False,False,False,False,False,False,11644.999997,2005.000000,2005.000000,-0.000006,-2.059479e-08,434.000002,1733.999988,1734.000013,0.000004,1734.000010,2005.000001,659.999989,159.999999,2.400000e+01,12.468437
2,1298,180,RM,35,Pave,#na#,Reg,Lvl,AllPub,Inside,Gtl,Edwards,Norm,Norm,TwnhsE,SFoyer,6,5,Gable,CompShg,VinylSd,VinylSd,BrkFace,TA,TA,PConc,Gd,TA,Gd,GLQ,547,Unf,GasA,Gd,Y,SBrkr,0,1,0,2,0,2,1,TA,5,Typ,0,#na#,Basment,Fin,2,TA,TA,Y,0,0,0,0,#na#,#na#,#na#,0,6,New,Partial,False,False,False,False,False,False,False,False,False,False,False,False,False,False,3675.000119,2005.000000,2005.999999,82.000000,-2.059479e-08,-0.000012,547.000026,1072.000000,0.000004,1072.000005,2005.000001,524.999999,-0.000003,4.400000e+01,11.849398
3,64,70,RM,50,Pave,#na#,IR1,Bnk,AllPub,Inside,Gtl,OldTown,RRAn,Feedr,1Fam,2Story,7,6,Gable,CompShg,Stucco,Stucco,None,TA,TA,BrkTil,TA,TA,No,Unf,0,Unf,GasA,Gd,Y,SBrkr,0,0,0,2,0,3,1,TA,9,Typ,0,#na#,Detchd,Unf,2,TA,TA,Y,64,0,0,0,#na#,GdPrv,#na#,0,4,WD,Normal,False,False,False,False,False,False,False,False,False,False,False,False,False,False,10300.000041,1920.999999,1950.000001,-0.000006,-2.059479e-08,576.000001,575.999980,902.000008,807.999992,1710.000011,1990.000000,480.000001,12.000003,1.100000e+01,11.849398
4,884,75,RL,60,Pave,#na#,Reg,Bnk,AllPub,Inside,Gtl,SWISU,Norm,Norm,1Fam,2.5Fin,4,5,Gable,CompShg,Wd Sdng,Wd Sdng,None,Gd,TA,PConc,TA,Fa,No,Unf,0,Unf,GasA,TA,N,SBrkr,481,1,0,1,0,5,1,TA,10,Typ,0,#na#,Detchd,Unf,1,TA,Gd,Y,0,0,0,0,#na#,#na#,#na#,0,3,WD,Normal,False,False,False,False,False,False,False,False,False,False,False,False,False,False,6204.000164,1912.000000,1950.000001,-0.000006,-2.059479e-08,795.000007,795.000003,954.000006,795.000013,2230.000010,1997.000000,440.000000,-0.000003,1.880000e+02,11.682669
5,521,190,RL,60,Pave,Grvl,Reg,Lvl,AllPub,Inside,Gtl,OldTown,Norm,Norm,2fmCon,2Story,4,7,Gable,CompShg,MetalSd,MetalSd,None,TA,TA,BrkTil,#na#,#na#,#na#,#na#,0,#na#,GasA,TA,N,FuseA,0,0,0,2,0,3,2,TA,7,Typ,0,#na#,#na#,#na#,0,#na#,#na#,N,210,0,0,0,#na#,#na#,#na#,0,8,WD,Normal,False,False,False,False,False,False,False,False,False,False,False,False,False,False,10800.000002,1900.0

In [ ]:
loss_fun = torch.nn.functional.mse_loss
learn = tabular_learner(dls, n_out=1, loss_func=loss_fun, config=dict(active_fun=Mish))

In [ ]:
learn.load("model-final")

In [ ]:
df_s = pd.read_csv(clean_data_path, encoding="utf8")

## Feature importance

In [ ]:
dls.valid.xs

,Id,MSSubClass,MSZoning,LotFrontage,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,GrLivArea,GarageYrBlt,GarageArea,WoodDeckSF,OpenPorchSF
80,0,8,2,0,1,0,4,3,1,4,...,-0.219295,-0.351932,-0.147713,-0.538062,-0.699373,-1.127443,-0.005901,-0.696164,0.931444,-0.784382
81,0,1,2,21,1,0,4,3,1,4,...,-0.219295,-0.630575,1.229569,1.189541,-0.699373,0.371898,1.363213,1.039508,0.208879,1.187216
82,0,6,2,17,1,0,4,3,1,4,...,-0.219295,0.495607,-0.659466,-1.179982,1.357005,0.144212,-1.752702,-0.863458,0.183963,-0.784382
83,0,2,3,11,1,0,4,3,1,4,...,-0.219295,0.381828,0.114838,-0.208729,-0.699373,-0.841623,-0.053112,-0.110636,0.566009,1.366452
84,0,1,2,0,1,0,4,3,1,4,...,-0.219295,0.193744,1.472096,1.493756,-0.699373,0.635918,1.268791,2.064182,-0.812678,-0.318368
85,0,2,2,0,1,0,4,3,1,4,...,-0.219295,0.637250,-0.523740,0.664841,-0.699373,-0.083475,1.410424,0.600362,-0.812678,-0.784382
86,0,8,2,0,1,0,1,3,1,2,...,-0.219295,1.417450,1.398670,1.524456,-0.699373,0.662562,-0.478010,0.098481,-0.812678,-0.784382
87,0,1,2,23,1,0,4,0,1,4,...,-0.219295,-0.235831,-0.203338,-0.607836,-0.699373,-1.187998,-0.572431,-0.152459,0.183963,-0.784382
88,0,10,3,3,1,0,4,3,1,3,...,-0.219295,-0.516795,0.366265,0.106649,-0.699373,-0.567915,0.985526,-0.309297,-0.812678,1.456070
89,0,4,2,0,1,0,4,3,1,1,...,1.419566,-0.282271,-0.470340,-0.942751,0.755072,-0.185207,-0.478010,-0.643885,-0.812678,-0.784382


In [ ]:
def get_df_preds(df:pd.DataFrame, learn:Learner, cat_names:List[str], cont_names:List[str]):
    "`df` is the `TabularPandas.valid.xs` attribute"
    x_cat = torch.from_numpy(df[cat_names].values).long()
    x_cont = torch.from_numpy(df[cont_names].values).float()
    learn.model.eval()
    return learn.model.forward(x_cat, x_cont)

In [ ]:
%%time
get_df_preds(to.valid.xs, learn, to.cat_names, to.cont_names)

CPU times: user 15.6 ms, sys: 0 ns, total: 15.6 ms
Wall time: 9.14 ms


tensor([[10.2952],
        [11.1078],
        [11.5308],
        [ 8.7183],
        [11.1293],
        [12.4043],
        [ 7.8074],
        [10.0069],
        [ 5.1600],
        [11.6227],
        [10.9821],
        [ 7.3126],
        [12.2341],
        [ 9.7876],
        [ 9.0370],
        [13.0619],
        [10.4402],
        [10.5533],
        [ 9.8991],
        [10.1030]], grad_fn=<AddmmBackward>)

In [ ]:
%%time
learn.get_preds(dl=dls.valid)

CPU times: user 78.1 ms, sys: 0 ns, total: 78.1 ms
Wall time: 33.7 ms


(tensor([[10.2952],
         [11.1078],
         [11.5308],
         [ 8.7183],
         [11.1293],
         [12.4043],
         [ 7.8074],
         [10.0069],
         [ 5.1600],
         [11.6227],
         [10.9821],
         [ 7.3126],
         [12.2341],
         [ 9.7876],
         [ 9.0370],
         [13.0619],
         [10.4402],
         [10.5533],
         [ 9.8991],
         [10.1030]]),
 tensor([[11.9177],
         [12.3014],
         [11.5899],
         [11.6128],
         [12.5242],
         [11.6263],
         [12.1955],
         [11.4590],
         [12.0076],
         [11.5617],
         [11.8123],
         [11.6263],
         [12.2308],
         [12.0137],
         [12.8610],
         [12.0725],
         [11.6656],
         [11.5617],
         [12.4684],
         [11.6527]]))

In [ ]:
def get_feature_importance(to:TabularPandas, learn:Learner):
    feature_losses = {}
    
    ref_loss = learn.loss_func(*learn.get_preds()).item()
    ys = torch.from_numpy(to.valid.ys.values).float()
    for col in to.valid.xs.columns:
        vals = to.valid.xs[col].values
        xs = to.valid.xs.copy()
        xs[col] = np.random.choice(vals)
        
        _ys = get_df_preds(xs, learn, to.cat_names, to.cont_names)
        
        feature_losses[col] = learn.loss_func(_ys, ys).item()
    
    return feature_losses, ref_loss

In [ ]:
%%time
importances, ref_loss = get_feature_importance(to, learn)

CPU times: user 3.31 s, sys: 78.1 ms, total: 3.39 s
Wall time: 865 ms


In [ ]:
# fig, ax = plt.subplots()
# ax.bar()
# plt.show()
losses = pd.DataFrame(importances.items(), columns=["column", "loss"])
losses["loss Δ"] = losses["loss"] - ref_loss

losses.sort_values("loss Δ", ascending=False, inplace=True)
display_all(losses)

,column,loss,loss Δ
11,Neighborhood,10.169829,3.438626
12,Condition1,8.758464,2.027260
92,OpenPorchSF,7.755461,1.024258
15,HouseStyle,7.655418,0.924214
44,TotRmsAbvGrd,7.640514,0.909311
50,GarageCars,7.429509,0.698305
30,BsmtFinSF1,7.383034,0.651830
21,Exterior2nd,7.349138,0.617935
20,Exterior1st,7.137808,0.406604
62,MoSold,7.105027,0.373823
